## Mount to Google Driver

In [1]:
mount_path = "MakeUP"
try:
    from google.colab import drive
    import os
    drive.mount('/content/gdrive/', force_remount=True)
    os.chdir(os.path.join('/content/gdrive/My Drive/',mount_path))
    print("save result at google driver")
except:
    print("save result at local")

Mounted at /content/gdrive/
save result at google driver


In [ ]:
! pip3 install .

## Check/install envs and import

In [3]:
from torchvision import datasets, transforms
from transformers import Trainer, TrainingArguments

from makeup.model.efficientnet import EfficientNetModify, imageclassify_collect_fn
from makeup.model.metrics import acc_metrics
from makeup.utils import logmanager
logger = logmanager.get_logger(__name__)

## Par Setting

In [4]:
image_size = 224
train_data_path = "dataset/train/"
valid_data_path = "dataset/valid/"

model_name = 'efficientnet-b4'

#### image process

In [5]:
# 準備訓練資料
# ImageFolder假設指定路徑下有多個資料夾，每個資料夾內為同一類的圖片，資料加名稱類別名
train_dataset = datasets.ImageFolder(train_data_path,
                                     transforms.Compose([
                                        transforms.Resize(image_size),
                                        transforms.CenterCrop(image_size),
                                        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                     ]))

In [6]:
# 準備訓練資料
# ImageFolder假設指定路徑下有多個資料夾，每個資料夾內為同一類的圖片，資料加名稱類別名
valid_dataset = datasets.ImageFolder(valid_data_path,
                                     transforms.Compose([
                                        transforms.Resize(image_size),
                                        transforms.CenterCrop(image_size),
                                        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                     ]))

#### modify EfficientNet

In [7]:
model_1 = EfficientNetModify.from_pretrained(model_name, num_classes = 4, include_top=False)

Loaded pretrained weights for efficientnet-b4


#### Trainer

In [8]:
training_args = TrainingArguments(
                                    output_dir="./results",  # 輸出模型的資料夾
                                    num_train_epochs=50,          # 訓練代數
                                    per_device_train_batch_size=32,    # train時的batch size
                                    per_device_eval_batch_size=32,    # eval時的batch size
                                    gradient_accumulation_steps=1,    # 每幾個batch update一次參數
                                    load_best_model_at_end=True, # 訓練完後自動讀取最佳的model, trainer 會忽略 save_strategy,save_steps 在每次eval後存模型
                                    warmup_steps=500,          # 前幾個batch要做warm up
                                    weight_decay=0.00001,          # learning rate decay
                                    eval_steps=50,            # 每幾個step要eval 預設500
                                    # save_steps=50,            # 每幾個step要save 預設500
                                    logging_steps=1,         
                                    evaluation_strategy="steps",   # 用STEPS來判斷是否要eval
                                    dataloader_num_workers=2,      # 開幾個CPU做dataloader
                                 )

In [9]:
# get trainer
trainer = Trainer(
                    model=model_1,
                    args=training_args,
                    train_dataset = train_dataset,
                    eval_dataset = valid_dataset,
                    data_collator = imageclassify_collect_fn,
                    compute_metrics = acc_metrics
                  )

In [ ]:
trainer.train()

## Show Data

In [ ]:
modelpath = "./results/checkpoint-500/pytorch_model.bin"

model = EfficientNetModify.from_pretrained(model_name, weights_path=modelpath, num_classes = 4, include_top=False, load_fc=True)

# get trainer
trainer = Trainer(
                    model=model,
                    args=training_args,
                    train_dataset = train_dataset,
                    eval_dataset = valid_dataset,
                    data_collator = imageclassify_collect_fn,
                    compute_metrics = acc_metrics
                  )

In [ ]:
predict = trainer.predict(valid_dataset)

In [ ]:
valid_dataset.classes

In [ ]:
from PIL import Image
from IPython.display import display
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
plt.show()

def desplay_image(path):
    im = Image.open(path)
    im = im.resize((100, 100))
    if envchecker.is_in_notebook():
        display(im)
    else:
        im.show() 

def show_confusion_matrix(true, predict, x=True, y=True):
    cm = confusion_matrix(true, predict)
    sns.heatmap(cm, xticklabels=x, yticklabels=y)
    plt.show()


def show_result_classfyimg(predict, dataset):
    metric = predict.metrics
    predict_labels = predict.predictions[0].argmax(-1)
    labels = predict.label_ids
    index = 0
    print("------------ Metrics ------------")
    for k in predict.metrics:
        print("{} : {}".format(k, predict.metrics[k]))

    print("------------ Confuse metric ------------")
    show_confusion_matrix(labels, predict_labels, dataset.classes, dataset.classes)

    print("------------ Show error img ------------")
    for predict_label, label, sample in zip(predict_labels,labels,dataset.samples):
        if predict_label != label:
            print("label {}".format(dataset.classes[label]))
            print("predict {}".format(dataset.classes[predict_label]))
            desplay_image(sample[0]) 
            print()
        index+=1

In [ ]:
show_result_classfyimg(predict, valid_dataset)